In [205]:
import modern_robotics as mr
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols, mechanics_printing
mechanics_printing()

In [206]:
def exp3(omega, theta):
    omega = sp.skew(omega)
    R = sp.eye(3) + sp.sin(theta) * omega + (1 - sp.cos(theta)) * omega * omega
    return R

def Ad(T):
    AdT = sp.zeros(6)
    R = sp.Matrix(T[:3, :3])
    AdT[:3, :3] = R
    AdT[3:, 3:] = R
    AdT[3:, :3] = sp.skew(T[:3, 3]) * R
    return AdT

def rotX(alfa_im1):
    Rx = sp.eye(4)
    Rx[1,1] =    sp.cos(alfa_im1)
    Rx[1,2] =   -sp.sin(alfa_im1)
    Rx[2,1] =    sp.sin(alfa_im1)
    Rx[2,2] =    sp.cos(alfa_im1)
    return Rx

def rotZ(alfa_im1):
    Rz = sp.eye(4)
    Rz[0,0] =    sp.cos(alfa_im1)
    Rz[0,1] =   -sp.sin(alfa_im1)
    Rz[1,0] =    sp.sin(alfa_im1)
    Rz[1,1] =    sp.cos(alfa_im1)
    return Rz

def transX(a_im1):
    trA = sp.eye(4)
    trA[0,3] =  a_im1
    return trA

def transZ(d_i):
    trA = sp.eye(4)
    trA[2,3] =  d_i
    return trA


In [207]:
# Oppg 1
print("Task 1:")

th1, th2, th3, th4, th5, th6 = dynamicsymbols('th1, th2, th3, th4, th5, th6')

config = sp.Matrix([[0,0,0,th1],[sp.pi/2,0,0,th2],[0,455,0,th3],[sp.pi/2, 35, 420, th4],[sp.pi/2,0,0,th5],[sp.pi/2, 0,0,th6]])
config

Task 1:


⎡0   0    0   th₁⎤
⎢                ⎥
⎢π               ⎥
⎢─   0    0   th₂⎥
⎢2               ⎥
⎢                ⎥
⎢0  455   0   th₃⎥
⎢                ⎥
⎢π               ⎥
⎢─  35   420  th₄⎥
⎢2               ⎥
⎢                ⎥
⎢π               ⎥
⎢─   0    0   th₅⎥
⎢2               ⎥
⎢                ⎥
⎢π               ⎥
⎢─   0    0   th₆⎥
⎣2               ⎦

In [208]:
# oppg 2
Mi = sp.Matrix([[sp.eye(4)]*6])

for i in range(6):
    Mi[:,4*i:4*(i+1)] = rotX(config[i,0]) * transX(config[i,1]) * transZ(config[i,2])

M = sp.eye(4)
for n in range(5,-1,-1):
    M = Mi[:,4*n:4*(n+1)] * M

M

⎡1  0  0  490 ⎤
⎢             ⎥
⎢0  1  0   0  ⎥
⎢             ⎥
⎢0  0  1  -420⎥
⎢             ⎥
⎣0  0  0   1  ⎦

In [209]:
# oppg 3
Ai = sp.Matrix([[0,-1,0,0],[1,0,0,0],[0,0,0,0],[0,0,0,0]]) # This is a given matrix due to revolute joints

S_sp = sp.zeros(6)
for i in range(6):
    dot_sum = sp.eye(4)
    for n in range(i,-1,-1):
        dot_sum = Mi[:,4*n:4*(n+1)] * dot_sum
    S_skew = dot_sum * Ai * sp.Inverse(dot_sum)
    S_sp[0,i] = S_skew[2,1] 
    S_sp[1,i] = S_skew[0,2] 
    S_sp[2,i] = S_skew[1,0] 
    S_sp[3,i] = S_skew[0,3] 
    S_sp[4,i] = S_skew[1,3] 
    S_sp[5,i] = S_skew[2,3] 

S_sp

⎡0  0    0     0    0    0  ⎤
⎢                           ⎥
⎢0  -1   -1    0    1    0  ⎥
⎢                           ⎥
⎢1  0    0    -1    0    1  ⎥
⎢                           ⎥
⎢0  0    0     0   420   0  ⎥
⎢                           ⎥
⎢0  0    0    490   0   -490⎥
⎢                           ⎥
⎣0  0   -455   0   490   0  ⎦

In [211]:
# oppg 4

# Hvordan finner vi S_bs? antar ved bruk av T_bs, men hvordan finner vi den uten å gjøre det hele på nytt? 